## Module 2 Project 

link to spreadsheet: https://docs.google.com/spreadsheets/d/1kL1rU6_G7Klr78RxtadjEI4dSzmYr3hZEPXT-HaghnA/edit#gid=453759461

link to github: https://github.com/petehumphries/Flatiron_School-Module_2_Proj.git


### Executive Summary

This notebook is split into the following sections:

    1) Connect to the SQL database
    2) Extract + Transform + Load
    3) 


Upon completion of this lab, each unique team in this dataset should have a record in the MongoDB instance containing the following information:

The name of the team
The total number of goals scored by the team during the 2011 season
The total number of wins the team earned during the 2011 season
A histogram visualization of the team's wins and losses for the 2011 season (store the visualization directly)

The team's win percentage on days where it was raining during games in the 2011 season.

### Inputs

    All team names
    All Match results for 2011
    All goals scored for 2011
    All match dates 
    
    Weather

SQL DB: Name of team, goals scored during the 2011 season, total number of wins
Weather: From the Weather API - Berlin only
Join by date

List the match days => max = 365, likely below 200

### Tables:

There are the following Tables: 

Matches, Teams, Unique Teams, Teams_in_Matches, FlatView, FlatView_Advanced, FlatView_Chrono_TeamOrder_Reduced,  

Match_ID (int): unique ID per match
Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
Season (int): Season the match took place in (usually covering the period of August till May of the following year)
Date (str): Date of the match
HomeTeam (str): Name of the home team
AwayTeam (str): Name of the away team
FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

#### Table: Teams

Season (str): Football season for which the data is valid
TeamName (str): Name of the team the data concerns
KaderHome (str): Number of Players in the squad
AvgAgeHome (str): Average age of players
ForeignPlayersHome (str): Number of foreign players (non-German, non-English respectively) playing for the team
OverallMarketValueHome (str): Overall market value of the team pre-season in EUR (based on data from transfermarkt.de)
AvgMarketValueHome (str): Average market value (per player) of the team pre-season in EUR (based on data from transfermarkt.de)
StadiumCapacity (str): Maximum stadium capacity of the team's home stadium

#### Table: Unique Teams

TeamName (str): Name of a team
Unique_Team_ID (int): Unique identifier for each team

#### Table: Teams_in_Matches

Match_ID (int): Unique match ID
Unique_Team_ID (int): Unique team ID (This table is used to easily retrieve each match a given team has played in)
Based on these tables I created a couple of views which I used as input for my machine learning models:

#### View: FlatView

Combination of all matches with the respective additional data from Teams table for both home and away team.

#### View: FlatView_Advanced

Same as Flatview but also includes Unique_Team_ID and Unique_Team in order to easily retrieve all matches played by a team in chronological order.

#### View: FlatView_Chrono_TeamOrder_Reduced

Similar to Flatview_Advanced, however missing the additional attributes from team in order to have a longer history including years 1993 - 2004. Especially interesting if one is only interested in analyzing winning/loosing streaks.
Match_ID (int): unique ID per match
Div (str): identifies the division the match was played in (D1 = Bundesliga, D2 = Bundesliga 2, E0 = English Premier League)
Season (int): Season the match took place in (usually covering the period of August till May of the following year)
Date (str): Date of the match
HomeTeam (str): Name of the home team
AwayTeam (str): Name of the away team
FTHG (int) (Full Time Home Goals): Number of goals scored by the home team
FTAG (int) (Full Time Away Goals): Number of goals scored by the away team
FTR (str) (Full Time Result): 3-way result of the match (H = Home Win, D = Draw, A = Away Win)

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import json

conn = sqlite3.connect('database.sqlite')
cur = conn.cursor()

Extracting all games from the Matches table, filtering on the 2011 season

Excluding the Enplish Premiership Games from the import using AND Div <> 'E0' condition. 612 entries

In [2]:
cur.execute("""SELECT * FROM MATCHES WHERE Season == '2011' AND Div <> 'E0';""")

In [3]:
len(cur.fetchall()) # 612 rows expected

612

To exclude the English Premier League Matches where Div = E0

In [4]:
cur.execute("""SELECT * FROM MATCHES WHERE Season == '2011' AND Div <> 'E0';""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head(1000)

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D
...,...,...,...,...,...,...,...,...,...
607,1699,D2,2011,2011-07-24,Paderborn,Fortuna Dusseldorf,1,1,D
608,1700,D2,2011,2011-12-10,Bochum,Fortuna Dusseldorf,1,1,D
609,1701,D2,2011,2011-08-15,Ein Frankfurt,Fortuna Dusseldorf,1,1,D
610,1702,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D


Creating a new column to show "Match_Winner" 

In [5]:
col         = 'FTR'
conditions  = [ df[col] == 'H', df[col] == 'A', df[col] == 'D' ]
choices     = [ df['HomeTeam'], df['AwayTeam'], 'Draw' ]
df["Match_Winner"] = np.select(conditions, choices, default=np.nan) 

In [6]:
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Match_Winner
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich
1,1093,D1,2011,2011-12-11,Stuttgart,Bayern Munich,1,2,A,Bayern Munich
2,1094,D1,2011,2011-08-13,Wolfsburg,Bayern Munich,0,1,A,Bayern Munich
3,1095,D1,2011,2011-11-27,Mainz,Bayern Munich,3,2,H,Mainz
4,1096,D1,2011,2012-02-18,Freiburg,Bayern Munich,0,0,D,Draw
...,...,...,...,...,...,...,...,...,...,...
607,1699,D2,2011,2011-07-24,Paderborn,Fortuna Dusseldorf,1,1,D,Draw
608,1700,D2,2011,2011-12-10,Bochum,Fortuna Dusseldorf,1,1,D,Draw
609,1701,D2,2011,2011-08-15,Ein Frankfurt,Fortuna Dusseldorf,1,1,D,Draw
610,1702,D2,2011,2011-08-27,Aachen,Fortuna Dusseldorf,0,0,D,Draw


In [7]:
match_dates_unique=df['Date'].unique()
len(match_dates_unique)

136

Selecting the unique dates of football matches to send weather requestes for. Want to minimise the requests

In [8]:
match_dates = np.sort(match_dates_unique)
match_dates

array(['2011-07-15', '2011-07-16', '2011-07-17', '2011-07-18',
       '2011-07-22', '2011-07-23', '2011-07-24', '2011-07-25',
       '2011-08-05', '2011-08-06', '2011-08-07', '2011-08-08',
       '2011-08-12', '2011-08-13', '2011-08-14', '2011-08-15',
       '2011-08-19', '2011-08-20', '2011-08-21', '2011-08-22',
       '2011-08-26', '2011-08-27', '2011-08-28', '2011-08-29',
       '2011-09-09', '2011-09-10', '2011-09-11', '2011-09-12',
       '2011-09-16', '2011-09-17', '2011-09-18', '2011-09-19',
       '2011-09-23', '2011-09-24', '2011-09-25', '2011-09-26',
       '2011-09-30', '2011-10-01', '2011-10-02', '2011-10-03',
       '2011-10-14', '2011-10-15', '2011-10-16', '2011-10-17',
       '2011-10-21', '2011-10-22', '2011-10-23', '2011-10-28',
       '2011-10-29', '2011-10-30', '2011-10-31', '2011-11-04',
       '2011-11-05', '2011-11-06', '2011-11-07', '2011-11-18',
       '2011-11-19', '2011-11-20', '2011-11-21', '2011-11-25',
       '2011-11-26', '2011-11-27', '2011-11-28', '2011-

Build a function to turn match days in to an array

In [9]:
def weather_required_dates(match_dates):
    weather_req_dates = []
    for d in match_dates:
        weather_req_dates.append(d)
    return weather_req_dates

In [53]:
match_days = weather_required_dates(match_dates)
df_match_days=pd.DataFrame(match_days)
df_match_days.columns = ['Date']
df_match_days.head(2)

,Date
0,2011-07-15
1,2011-07-16


### Dark Skies API

Data requirement from Dark Skies AP

Finally, let's get our client id and API key into our jupyter notebook.
If we remember that our file is just a regular JSON file, open the file and pull out the appropriate information from the ~/.secret/yelp_api.json file.

The API documentation can be found at: https://darksky.net/dev/docs

The Co-ordinates for Berlin sourced from https://latitude.to/map/de/germany/cities/berlin are:

DD 52.52437 13.41053

DMS 52°31'27.73" N 13°24'37.91" E

In [11]:
import json

#path = "~/.secret/dark_sky_api.txt"
path = 'c://Users/User1/.secret/dark_sky_api.txt'

def get_keys(path):
    with open(path) as f:
        return json.load(f)

In [12]:
keys = get_keys(path)
#print(keys)

In [13]:
api_key = keys['api_key']

The API documentation can be found at: https://darksky.net/dev/docs

In [14]:
latitude = 52.52437 
longitude = 13.41053

import requests
SEARCH_LIMIT = 1
url = 'https://api.darksky.net/forecast/'+api_key+'/'+ str(latitude) +','+ str(longitude)
response = requests.get(url)

First test response from the API using the "currently" default and longitude & latitude

In [15]:
#https://api.darksky.net/forecast/[api_key]/[latitude],[longitude]
print(response)
print(type(response.text))
print(response.text[:500])

<Response [200]>
<class 'str'>
{"latitude":52.52437,"longitude":13.41053,"timezone":"Europe/Berlin","currently":{"time":1580459091,"summary":"Overcast","icon":"cloudy","precipIntensity":0.0022,"precipProbability":0.17,"precipType":"rain","temperature":47.92,"apparentTemperature":41.65,"dewPoint":44.21,"humidity":0.87,"pressure":1002.7,"windSpeed":16.36,"windGust":32.92,"windBearing":277,"cloudCover":0.93,"uvIndex":0,"visibility":10,"ozone":286},"hourly":{"summary":"Light rain starting tomorrow morning.","icon":"rain","data":[


### Time Machine Request

We need to return one weather request per match date. DarkSky uses the unix date convention, so match dates need to be converted in UNIX DATE FORMAT. Running with a test date

In [16]:
date = '2011-07-17'
url = 'https://api.darksky.net/forecast/'+api_key+'/'+ str(latitude) +','+ str(longitude) + ',' +str(date) + 'T16:00:00'
url

'https://api.darksky.net/forecast/cc668e0d95b3fa324b37c548fe2b5731/52.52437,13.41053,2011-07-17T16:00:00'

In [54]:
test_response = requests.get(url)
print(test_response)
print(type(test_response.text))
print(test_response.text[:500])

<Response [200]>
<class 'str'>
{"latitude":52.52437,"longitude":13.41053,"timezone":"Europe/Berlin","currently":{"time":1310911200,"precipIntensity":0,"precipProbability":0,"temperature":79.39,"apparentTemperature":79.39,"dewPoint":55.79,"humidity":0.44,"windSpeed":14.96,"windGust":14.99,"windBearing":230,"visibility":6.206},"hourly":{"summary":"Rain overnight and in the evening.","icon":"rain","data":[{"time":1310853600,"precipIntensity":0,"precipProbability":0,"temperature":65.79,"apparentTemperature":65.79,"dewPoint":50.83


### Create a Weather Class: OOP Requirement 1

In [42]:
class WeatherGetter():
# The function works is slim and only runs once 

    def __init__(self):    

        self.latitude = 52.52437 
        self.longitude = 13.41053
        
    def weather_retriever(self, match_days):
        weather = []
        for date in match_days:
            url = 'https://api.darksky.net/forecast/'+api_key+'/'+ str(self.latitude) +','+ str(self.longitude) + ',' +str(date) + 'T16:00:00'
            response = requests.get(url)
            weather.append(response.json()['daily']['data'][0].get('icon','clear'))
        return(weather)

Test the Class on a single date before sending all match dates to the API 

In [55]:
test_single_run = weather_testing.weather_retriever(['2011-07-15'])

In [56]:
test_single_run

['partly-cloudy-day']

In [50]:
weather = weather_testing.weather_retriever(match_days)

In [51]:
len(weather)

136

In [19]:
weather

Save the weather output down as an emergency fallback in case of API issues later!!!!!!!!!!

In [61]:
df_weather=pd.DataFrame(weather)
df_weather.to_csv('weather_on_match_days_berlin')

In [21]:
weather

['partly-cloudy-day',
 'clear',
 'rain',
 'rain',
 'rain',
 'rain',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'rain',
 'rain',
 'rain',
 'rain',
 'rain',
 'partly-cloudy-day',
 'rain',
 'rain',
 'partly-cloudy-day',
 'clear',
 'clear',
 'partly-cloudy-day',
 'clear',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'rain',
 'rain',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'rain',
 'rain',
 'partly-cloudy-day',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'clear',
 'partly-cloudy-day',
 'clear',
 'clear',
 'partly-cloudy-day',
 'clear',
 'clear',
 'partly-cloudy-day',
 'clear',
 'clear',
 'clear',
 'clear',
 'cloudy',
 'partly-cloudy-day',
 'clear',
 'clear',
 'rain',
 'rain',
 'wind',
 'partly-cloudy-day',
 'rain',
 'rain',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'partly-cloudy-day',
 'rain',
 'rain',
 'rain',
 'ra

Add weather to df_match_ days vector to build a match day vs weather dictionary / key value / pair 

In [22]:
df_match_days['Weather'] = weather

In [23]:
#df_match_days.columns = ['Date']#, 'Weather']
df_match_days

,Date,Weather
0,2011-07-15,partly-cloudy-day
1,2011-07-16,clear
2,2011-07-17,rain
3,2011-07-18,rain
4,2011-07-22,rain
...,...,...
131,2012-04-23,clear
132,2012-04-28,clear
133,2012-04-29,clear
134,2012-05-05,rain


Join / Merge the weather information to the results dataframe 

In [24]:
df = pd.merge(df,df_match_days)

In [150]:
df

,Match_ID,Div,Season,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,Match_Winner,Weather
0,1092,D1,2011,2012-03-31,Nurnberg,Bayern Munich,0,1,A,Bayern Munich,rain
1,1189,D1,2011,2012-03-31,Augsburg,FC Koln,2,1,H,Augsburg,rain
2,1206,D1,2011,2012-03-31,Leverkusen,Freiburg,0,2,A,Freiburg,rain
3,1215,D1,2011,2012-03-31,Kaiserslautern,Hamburg,0,1,A,Hamburg,rain
4,1396,D2,2011,2012-03-31,Munich 1860,Hansa Rostock,0,1,A,Hansa Rostock,rain
...,...,...,...,...,...,...,...,...,...,...,...
607,1689,D2,2011,2011-12-05,Duisburg,Fortuna Dusseldorf,0,2,A,Fortuna Dusseldorf,partly-cloudy-day
608,1690,D2,2011,2012-04-16,Dresden,Fortuna Dusseldorf,2,1,H,Dresden,clear
609,1694,D2,2011,2011-10-03,Braunschweig,Fortuna Dusseldorf,1,1,D,Draw,clear
610,1696,D2,2011,2012-03-19,Cottbus,Fortuna Dusseldorf,1,1,D,Draw,partly-cloudy-day


Save the dataframe to csv in case of need to rollback

In [151]:
df.to_csv('win_lose_draw_rain.csv')

In [152]:
home = df['HomeTeam'].unique()
len(home)

36

In [153]:
away = df['AwayTeam'].unique()
len(away)

36

Start the process of creating team name, wins, losses, percentage, rain wins percentage

In [156]:
home_goals_sum = df['FTHG'].sum() # 996 home goals
home_goals_sum

996

In [157]:
away_goals_sum = df['FTAG'].sum() # 734 away goals
away_goals_sum

734

In [158]:
len(df['Match_ID'].unique()) # 612 unique match ids

612

In [159]:
h_teams = df['HomeTeam'].unique()
len(h_teams)

36

In [162]:
home_goals = df.groupby(['HomeTeam'])['FTHG'].sum()
home_goals=pd.DataFrame(home_goals)
home_goals = home_goals.reset_index()
home_goals.columns = ['Team', 'FTHG']

away_goals = df.groupby(['AwayTeam'])['FTAG'].sum()
away_goals=pd.DataFrame(away_goals)
away_goals = away_goals.reset_index()
away_goals.columns = ['Team', 'FTAG']


In [166]:
away_goals.head(2)

,Team,FTAG
0,Aachen,15
1,Augsburg,16


In [167]:
home_goals.head(2)

,Team,FTHG
0,Aachen,15
1,Augsburg,20


In [168]:
total_goals = pd.merge(home_goals, away_goals)

In [170]:
total_goals.head()

,Team,FTHG,FTAG
0,Aachen,15,15
1,Augsburg,20,16
2,Bayern Munich,49,28
3,Bochum,23,18
4,Braunschweig,21,16


In [172]:
total_goals['FTTG'] = total_goals['FTHG'] + total_goals['FTAG']
total_goals.head()

,Team,FTHG,FTAG,FTTG
0,Aachen,15,15,30
1,Augsburg,20,16,36
2,Bayern Munich,49,28,77
3,Bochum,23,18,41
4,Braunschweig,21,16,37


In [ ]:
class win_lose_draw_stats():
    
    def _init_(self, df = None, filter_cond = None):
        self.name = name
        self.df = df
        self.filter_cond = filter_cond
    
    def goals(self):
        home_team_goals = self.df.loc[:, ['HomeTeam', 'FTHG']].groupby(['HomeTeam']).sum()
        away_team_goals = self.df.loc[:, ['AwayTeam', 'FTAG']].groupby(['AwayTeam']).sum()
        team_goals = pd.concat([home_team_goals, away_team_goals], axis =  1)
        team_goals['total_goals'] = team_goals['FTHG'] + team_goals['FTAG']
        return team_goals
    
    def wins(self):
        home_team_wins = self.df.loc[:, ['HomeTeam', 'FTR_H']].groupby(['HomeTeam']).sum()
        away_team_wins = self.df.loc[:, ['AwayTeam', 'FTR_A']].groupby(['AwayTeam']).sum()
        team_wins = pd.concat([home_team_wins, away_team_wins], axis =  1)
        team_wins['total_wins'] = team_wins['FTR_H'] + team_wins['FTR_A']
        return team_wins
    
    def losses(self):
        home_team_losses = self.df.loc[:, ['HomeTeam', 'FTR_A']].groupby(['HomeTeam']).sum()
        away_team_losses = self.df.loc[:, ['AwayTeam', 'FTR_H']].groupby(['AwayTeam']).sum()
        team_losses = pd.concat([home_team_losses, away_team_losses], axis =  1)
        team_losses['total_losses'] = team_losses['FTR_A'] + team_losses['FTR_H']
        return team_losses
    
    def draws(self):
        home_team_draws = self.df.loc[:, ['HomeTeam', 'FTR_D']].groupby(['HomeTeam']).sum()
        away_team_draws = self.df.loc[:, ['AwayTeam', 'FTR_D']].groupby(['AwayTeam']).sum()
        home_team_draws = home_team_draws.rename(columns = {'FTR_D': 'FTR_D_H'})
        away_team_draws = away_team_draws.rename(columns = {'FTR_D': 'FTR_D_A'})
        team_draws = pd.concat([home_team_draws, away_team_draws], axis =  1)
        team_draws['total_draws'] = team_draws['FTR_D_H'] + team_draws['FTR_D_A']
        return team_draws
    
    def w_l_d(self):
        g = self.goals()
        w = self.wins()
        l = self.losses()
        d = self.draws()
        full_stats = pd.concat([g.loc[:, ['total_goals']],
                             w.loc[:, ['total_wins']],
                             l.loc[:, ['total_losses']],
                             d.loc[:, ['total_draws']]], axis = 1)
        full_stats['win_pct'] = full_stats['total_wins']/(full_stats['total_wins'] + full_stats['total_losses'] + full_stats['total_draws'])
        return full_stats

In [ ]:
b = win_lose_draw_stats()
c = b.wins()

In [ ]:
def _init_(self, df = None, filter_cond = None):
        self.name = name
        self.df = df
        self.filter_cond = filter_cond